<a href="https://colab.research.google.com/github/Jucicarla/analise-de-sentimento/blob/main/analise_sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação de pacotes

In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


#Importar pacotes

In [1]:
from textblob import TextBlob
import pandas as pd
import json
# Codigo para montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
arquivo = '/content/drive/MyDrive/dataset-v2.dat'
d = []
with open(arquivo) as infile:
  for line in infile:
    try:
      item = json.loads(line.strip())
      d.append(item)
    except:
      None

In [3]:
df = pd.DataFrame(d)

In [4]:
df

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,ratingValue,itemReviewed
0,Jantar,692856288,2344932,larinhavercosa222,"Comida maravilhosa, lugar do verdadeiro chicle...",pt,True,"São Carlos, SP",26/07/2019,07/2019,5,"{'name': 'Imperador dos Camarões', 'address': ..."
1,Uma boa experiência.,229208487,3720906,Pano P,"Comida boa, drinks legais e um ambiente bem ac...",pt,True,"Manaus, AM",16/09/2014,None,4,"{'name': 'Guacamole Natal', 'address': {'stree..."
2,Otimo self service,679002332,2248105,Magma100,"Excelente pra ir com a família! Ótima Comida, ...",pt,True,"Goiânia, GO",04/06/2019,07/2018,4,"{'name': 'Chica Pitanga', 'address': {'streetA..."
3,O sorvete mais tradicional do estado,355671519,8168302,Yurie M,Tomo sorvete na Visabor faz mais de 30 anos. S...,pt,False,"Aracaju, SE",15/03/2016,01/2016,5,"{'name': 'Sorvetes Vi Sabor', 'address': {'str..."
4,Melhor hamburger!,268741921,5395141,Debora_C85,O melhor hamburger que já comi! Excelente aten...,pt,False,"Rio de Janeiro, RJ",28/04/2015,04/2015,5,"{'name': 'Z Deli Sandwich Shop', 'address': {'..."
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Nova opção no Paseo!!!,672210394,17424548,Fabiano B,"Ainda não inaugurado oficialmente, está atende...",pt,True,"Porto Alegre, RS",07/05/2019,05/2019,5,"{'name': 'Boteco Maestro', 'address': {'street..."
99996,Bom,380904400,2440184,Chris L,"Bom atendimento, lugar bonito, comida muito go...",pt,True,"Cork, Irlanda",08/06/2016,01/2016,4,"{'name': 'Casa do Marquês', 'address': {'stree..."
99997,Bom bar,186111858,2392996,Guilherme P,O Rio Scenarium é uma das baladas mais famosas...,pt,False,"São Paulo, SP",27/11/2013,11/2013,4,"{'name': 'Rio Scenarium', 'address': {'streetA..."
99998,Me senti enganada...,152745909,1062290,LisianeCohem,"Fui por indicação e com boa expectativa!\n116,...",pt,True,"Porto Alegre, RS",21/02/2013,None,2,"{'name': 'Celeiro', 'address': {'streetAddress..."


#Análise de Sentimentos

Exemplo de comentário

In [ ]:
comentario = "Conheço o restaurante em várias cidades do Brasil. O de Manaus não foge muito da qualidade e beleza. Atendimento bom, ambiente bonito. Mas a comida não surpreendeu, aliás até decepcionou. Não foi aquele sabor que estamos acostumados quando vamos a este restaurante. Pode ter sido um fato isolado, mas."

Texto convertido para o formato TextBlob

In [ ]:
comentario_blob = TextBlob(comentario)

Comentário traduzido para Inglês (US)

In [ ]:
comentario_blob_us = comentario_blob.translate(to='us')

#Cálculo da subjetividade

O valor da subjetividade é um número real entre [0.0, 1.0], onde 0.0 o texto é indicado como muito objetivo e 1.0 o texto é indicado como muito subjetivo.

In [ ]:
for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
    print(sentence_br, ':', sentence.sentiment.subjectivity)

Conheço o restaurante em várias cidades do Brasil. : 0.0
O de Manaus não foge muito da qualidade e beleza. : 1.0
Atendimento bom, ambiente bonito. : 0.8
Mas a comida não surpreendeu, aliás até decepcionou. : 0.625
Não foi aquele sabor que estamos acostumados quando vamos a este restaurante. : 0.0
Pode ter sido um fato isolado, mas. : 0.0


#Cálculo da polaridade

O valor da polaridade é um número real entre os valores [-1.0, 1.0].

In [ ]:
for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
    print(sentence_br, ':', sentence.sentiment.polarity)

Conheço o restaurante em várias cidades do Brasil. : 0.0
O de Manaus não foge muito da qualidade e beleza. : -0.05
Atendimento bom, ambiente bonito. : 0.7749999999999999
Mas a comida não surpreendeu, aliás até decepcionou. : -0.55
Não foi aquele sabor que estamos acostumados quando vamos a este restaurante. : 0.0
Pode ter sido um fato isolado, mas. : 0.0
